# Abstract

Test out setting up test for whole spreadsheet run.

# Environment

In [1]:
from astropy.table import Table
import numpy as np

In [2]:
from jwst.lib import set_telescope_pointing as stp

# Library

In [3]:
def row_to_pointing(r, siaf=None):
    """Convert a row of the spreadsheet to a Pointing
    
    Parameters
    ----------
    r: dict
        A row of pointing information
        
    siaf: SIAF
        SIAF info to use. If None, a non-transformative
        SIAF will be used.
        
    Returns
    -------
    Pointing, vinfo_expected
        A `Pointing` instance filled out.
        A `WCS` instance with the expected V1 information.
    """
    if siaf is None:
        siaf = stp.SIAF()
        siaf.v2ref = 0.
        siaf.v3ref = 0.
        siaf.v3idlyang = 0.
        siaf.vparity = 1.

    q = np.asarray([
        r['SA_ZATTEST1'],        
        r['SA_ZATTEST2'],        
        r['SA_ZATTEST3'],        
        r['SA_ZATTEST4'],        
    ])
    
    j2fgs_matrix = np.asarray([
        r['SA_ZRFGS2J11'], r['SA_ZRFGS2J21'], r['SA_ZRFGS2J31'],
        r['SA_ZRFGS2J12'], r['SA_ZRFGS2J22'], r['SA_ZRFGS2J32'],
        r['SA_ZRFGS2J13'], r['SA_ZRFGS2J23'], r['SA_ZRFGS2J33'],
    ])
    
    fsmcorr = np.asarray([
        r['SA_ZADUCMDX'], r['SA_ZADUCMDY']
    ])
    
    p = stp.Pointing()
    p.q = q
    p.j2fgs_matrix = j2fgs_matrix
    p.fsmcorr = fsmcorr
    
    v = stp.WCSRef()
    v.ra = r['Vra']
    v.dec = r['Vdec']
    v.pa = r['V3PAatV1']
    
    return p, v

In [4]:
def wcs_equality(left, right):
    """Assert equality of to WCSRef instances"""
    assert np.isclose(left.ra, right.ra), 'RAs are different {} != {}'.format(left.ra, right.ra)
    assert np.isclose(left.dec, right.dec), 'DECs are different {} != {}'.format(left.dec, right.dec)
    assert np.isclose(left.pa, right.pa), 'PAs are different {} != {}'.format(left.pa, right.pa)

# Main

In [8]:
t = Table.read('../data/acs_tlm_data_4_stsci_dms_jitter_file_mod_RA_DEC_PA.csv')

In [9]:
pointings = [
    row_to_pointing(r)
    for r in t
]

In [10]:
pointings[0]

(Pointing(q=array([-0.36915286,  0.33763282,  0.05758533,  0.86395264]), j2fgs_matrix=array([-1.00444000e-03,  3.38145836e-03,  9.99993778e-01,  9.99999496e-01,
        -3.90000000e-14,  1.00444575e-03,  3.39649146e-06,  9.99994283e-01,
        -3.38145665e-03]), fsmcorr=array([0., 0.]), obstime=None),
 WCSRef(ra=348.927867, dec=-38.749239, pa=50.176708))

In [11]:
len(pointings)

20711

In [12]:
siaf = stp.SIAF()
siaf.v2ref = 0.
siaf.v3ref = 0.
siaf.v3idlyang = 0.
siaf.vparity = 1.

In [13]:
wcsinfo, vinfo = stp.calc_wcs(pointings[0][0], siaf)

In [14]:
vinfo

WCSRef(ra=348.92786699856623, dec=-38.749239233976155, pa=50.17670770067501)

In [15]:
pointings[0][1]

WCSRef(ra=348.927867, dec=-38.749239, pa=50.176708)

In [17]:
wcs_equality(vinfo, pointings[0][1])

In [19]:
for idx, pv in enumerate(pointings):
    p, v = pv
    wcsinfo, vinfo = stp.calc_wcs(p, siaf)
    try:
        wcs_equality(vinfo, v)
    except AssertionError as e:
        print('{}'.format(idx))
        print(e)
            

2885
PAs are different 50.33318902996746 != 50.334541
2886
PAs are different 50.333190141700285 != 50.334543
2887
PAs are different 50.33319100614259 != 50.334543
2888
PAs are different 50.333179832915334 != 50.334532
2889
PAs are different 50.33317671789264 != 50.334529
2890
PAs are different 50.333176448649716 != 50.334529
2891
PAs are different 50.33317844952174 != 50.334531
2892
PAs are different 50.333181155097705 != 50.334533
2893
PAs are different 50.33318097822715 != 50.334533
2894
PAs are different 50.333181449773924 != 50.334533
2895
PAs are different 50.333167500402354 != 50.334519
2896
PAs are different 50.333169779629614 != 50.334521
2897
PAs are different 50.33317161294328 != 50.334523
2898
PAs are different 50.33316242695892 != 50.334513
2899
PAs are different 50.33316388842605 != 50.334514
2900
PAs are different 50.33317132514329 != 50.334521
2901
PAs are different 50.33317350327447 != 50.334523
2902
PAs are different 50.3331731746676 != 50.334523
2903
PAs are different

3236
PAs are different 50.33349084899612 != 50.334059
3237
PAs are different 50.33349347617029 != 50.33406
3238
PAs are different 50.33349567406893 != 50.33406
3239
PAs are different 50.333498245351954 != 50.334061
3240
PAs are different 50.333500108784456 != 50.33406
3241
PAs are different 50.33350424091012 != 50.334062
3242
PAs are different 50.33350550693749 != 50.334061
3243
PAs are different 50.33349850933582 != 50.334052
3244
PAs are different 50.33349632618565 != 50.334048
3245
PAs are different 50.33350224851029 != 50.334051
3246
PAs are different 50.33350383510942 != 50.33405
3247
PAs are different 50.33350754120553 != 50.334052
3248
PAs are different 50.333510449793714 != 50.334052
3249
PAs are different 50.333513147839156 != 50.334053
3250
PAs are different 50.333514831855794 != 50.334052
3251
PAs are different 50.333512487881265 != 50.334048
3252
PAs are different 50.33351452018078 != 50.334048
3253
PAs are different 50.33351818913318 != 50.33405
3254
PAs are different 50.3